## Reinforcement Learning

Beim Reinforcement Learning werden im Gegensatz zu klassischen Machine Learning Methoden keine vorgegebenen Daten verwendet, sondern das Datenset wird während des Prozess generiert und direkt verarbeitet.

Zum RL gibt es mehrere Begriffe:

- Umgebung (Environment)
    - Erzeugt die Trainingsdaten indem sie das unterliegende Programm kontinuierlich ausliest und Inputs an das Programm weiter gibt
    - Effektiv das Programm, dass trainiert werden soll

- Prozess
    - Schleife die das Programm in Schritten ausführt
    - Inputs vom Agenten nehmen und weitergeben
 
- Agent
    - Kennt den Lernprozess
    - Agent generiert Inputs (Actions) und gibt diese an den Prozess weiter
    - Daraus lernt der Agent positive und negative Actions anhand der generierten Daten
    - Beim Lernen bekommt der Agent für positive Actions Punkte als Belohnung, die der Machine Learning Algorithmus als "Fortschritt" ansieht

Zudem benötigen wir noch einige Packages
- Gym(nasium)
    - Stellt Environment, Agent, Space, ... zur Verfügung
 
- Stable Baselines 3
    - Stellt einige nützliche Klassen und Funktion zum RL zur Verfügung

In [1]:
import gymnasium as gym

## Vordefinierte Umgebungen

Es gibt einige von OpenAI vorgegebene Testumgebungen dir wir zum testen verschiedener Methoden verwenden können

Eine davon ist die "CartPole" Umgebung

Bei dieser geht es darum, ein Kart mit einer Stange möglichst gerade zu halten durch Links- und Rechtsbewegungen

https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [11]:
env = gym.make("CartPole-v1", render_mode="human")

## Inhalte der Umgebung

Es gibt einige Begriffe die mit der Umgebung zusammenhängen

Spaces
- Action Space: Beschreibt die Aktionen die der Agent ausführen kann
- Observation Space: Liste der Daten die beim Ausführen von Actions in der Umgebung erzeugt werden -> Training mihilfe dieser Daten
- Discrete, Box, Tuple, Dict, ...

Funktionen der Umgebung
- __init__: Setzt die Umgebung auf (Setup)
- reset(): Setzt die Umgebung auf den Start zurück
- render(): Zeichne die Umgebung (falls möglich)
- step(): Führe die Action aus die vom Agenten ausgewählt wird
- close(): Beende die Umgebung

In [12]:
durchgänge = 5  # Anzahl der Durchgänge
for i in range(durchgänge):  # Teste die Umgebung X mal
    state = env.reset()  # Setze die Umgebung für den nächsten Durchlauf zurück
    done = False  # Brich den Prozess ab, wenn z.B. die Zeit abläuft
    score = 0

    while not done:
        env.render()  # Zeichne die Umgebung
        action = env.action_space.sample()  # Nimmt ein Random Element aus dem Action Space (Links oder Rechts)
        state, reward, done, term, info = env.step(action)  # state: Observation Space, reward: Belohnung, done: Fertig Ja/Nein, term: Termination (vorzeitiges Beenden), info: Extra Infos
        score += reward
    print(f"Durchgang {i}, Score: {score}")
env.close()

Durchgang 0, Score: 24.0
Durchgang 1, Score: 18.0
Durchgang 2, Score: 18.0
Durchgang 3, Score: 23.0
Durchgang 4, Score: 15.0


In [19]:
env.action_space.sample()  # Discrete(X): Aufsteigende Liste von 0 bis X

0

In [21]:
env.observation_space.sample()  # [Position, Geschwindigkeit, Winkel der Stange, L-R Geschwindigkeit der Stange]

array([-2.7179307e-01,  2.9223054e+38, -1.7964810e-01, -1.0291461e+38],
      dtype=float32)

## RL Model trainieren

Für ein RL Modell benötigen wir einen Algorithmus
- PPO: Proximal Policy Optimation
- https://stable-baselines.readthedocs.io/en/master/guide/algos.html

DummyVecEnv (optional)
- Simulation mehrerer Agenten gleichzeitig
- Sehr nützlich um Zeit zu sparen

Policy:
- Bestimmt wie der Algorithmus lernt
- https://stable-baselines.readthedocs.io/en/master/modules/policies.html

In [48]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [49]:
env = gym.make("CartPole-v1")
env = DummyVecEnv([lambda: env])
model = PPO("MlpPolicy", env, verbose=1)  # MLP: Multilayer Perceptron -> effektiv Dense Layer

Using cpu device


In [50]:
model.learn(total_timesteps=20000)

-----------------------------
| time/              |      |
|    fps             | 949  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 547         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008697905 |
|    clip_fraction        | 0.111       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00258    |
|    learning_rate        | 0.0003      |
|    loss                 | 5.69        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0173     |
|    value_loss           | 48.8        |
-----------------------------------------
----------------------------------

## Modell speichern

Wird benötigt, um das Modell auf die neue Umgebung anzuwenden

In [51]:
model.save("Models/CartPole")

## Modell evaluieren

In [52]:
from stable_baselines3.common.evaluation import evaluate_policy

In [53]:
evaluate_policy(model, env, n_eval_episodes=10)  # Durchschnittlicher Score, Standardabweichung

(500.0, 0.0)

In [54]:
env.close()

## Model testen

In [65]:
import numpy as np

In [66]:
env = gym.make("CartPole-v1")

In [71]:
model = PPO.load("Models/CartPoleModel", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [72]:
durchgänge = 5  # Anzahl der Durchgänge
for i in range(durchgänge):  # Teste die Umgebung X mal
    observation = env.reset()  # Setze die Umgebung für den nächsten Durchlauf zurück
    done = False  # Brich den Prozess ab, wenn z.B. die Zeit abläuft
    score = 0
    observation = observation[0].reshape(1, 4)

    while not done:
        env.render()  # Zeichne die Umgebung
        action, _ = model.predict(observation)  # Nimmt den State nach Reset
        if action.ndim == 1:
            action = action[0]
        observation, reward, done, term, info = env.step(action)  # state: Observation Space, reward: Belohnung, done: Fertig Ja/Nein, term: Termination (vorzeitiges Beenden), info: Extra Infos
        score += reward
    print(f"Durchgang {i}, Score: {score}")
env.close()

Durchgang 0, Score: 469.0
Durchgang 1, Score: 964.0
Durchgang 2, Score: 488.0
Durchgang 3, Score: 770.0
Durchgang 4, Score: 587.0
